In [ ]:
import numpy as np
import jax.numpy as jnp 
import jax
from jax import grad,jit,vmap
import tensorflow as tf #dataloader
tf.config.experimental.set_visible_devices([], "GPU")
from keras.utils import to_categorical
import matplotlib.pyplot as py
import timeit

In [ ]:
def NLL(x,y): #x =data, y= label one hot
  return -jnp.log(x[jnp.argmax(y)]) #assuming one hot

def resevoir(inputs,mask,res_params,res_bias): 
  res_params_m=res_params[0]#*mask[0]
  res1=jnp.dot(inputs,res_params_m.T) #+ res_bias[0]
  #res1=jax.nn.relu(res1)
  res2=res1
  for i in range(1,len(res_params)):
    res_params_m=res_params[i]#*mask[i]
    res2=jnp.dot(res2,res_params_m.T) #+ res_bias[0]
    res2=jax.nn.relu(res2)
  return res1,res2

def forward(inputs,mask,res_params,res_bias,params,bias): 
  
  res1,res2=resevoir(inputs,mask,res_params,res_bias)

  layer1=jnp.dot(res2,params[0].T) + bias[0] 
  layer1=jax.nn.relu(layer1) 
  layer2=jnp.dot(layer1,params[1].T)+bias[1]
  layer2=jax.nn.softmax(layer2)
  return layer2,res1,res2

v_forward=vmap(forward,in_axes=(0,None,None,None,None,None))
v_forward=jit(v_forward) 

def NLL_loss(inputs,target,mask,res_params,res_bias,params,bias): #loss function only accepts one sample at a time, however I will attempt to remedy this by vmap
  pred,_,_=forward(inputs,mask,res_params,res_bias,params,bias)
  final=NLL(pred,target)
  return final


#assumes the gradient input has shape [batch_size,layer]
def update_weights(params,bias, dparams,dbias ,lr=0.001): 
  for i in range(len(params)): #iterate through the layer. 
    params[i]=params[i]-(lr*jnp.sum(dparams[i],axis=0))
    bias[i]=bias[i]-(lr*jnp.sum(dbias[i],axis=0))
  return params,bias

def plasticity(layer,mask,interval=0.1): 
  layer_sum=jnp.sum(layer,axis=0) 
  #print(layer_sum.shape)
  for i in range(len(layer_sum)): 
    if layer_sum[i]<=0: 
      mask[i]=mask[i]-interval
      if mask[i][0]<interval: 
        mask[i]=0
    else: 
      mask[i]=mask[i]+interval
      


  return mask



gradient = grad(NLL_loss,argnums=(5,6))
v_gradient=vmap(gradient,in_axes=(0,0,None,None,None,None,None))

CIFAR10

In [ ]:
(train_data,train_label),(test_data,test_label) =tf.keras.datasets.cifar10.load_data() 

train_data=train_data.reshape(50000,1024,3)/255.0 
test_data=test_data.reshape(10000,1024,3)/255.0 
train_data=jnp.mean(train_data,axis=2)
test_data=jnp.mean(test_data,axis=2)

train_label=to_categorical(train_label) 
test_label=to_categorical(test_label) 

170500096/170498071 [==============================] - 3s 0us/step


### DMNIST dataset 

In [ ]:
(train_data,train_label),(test_data,test_label) =tf.keras.datasets.mnist.load_data() 
train_data=train_data.reshape(60000,784)/255.0 
test_data=test_data.reshape(10000,784)/255.0 
train_label=to_categorical(train_label) 
test_label=to_categorical(test_label) 

11493376/11490434 [==============================] - 0s 0us/step


###Second data

In [ ]:
(train_data_fash,train_label_fash),(test_data_fash,test_label_fash) =tf.keras.datasets.mnist.load_data() 
train_data_fash=train_data_fash.reshape(60000,784)/255.0 
test_data_fash=test_data_fash.reshape(10000,784)/255.0 
train_label_fash=to_categorical(train_label_fash) 
test_label_fash=to_categorical(test_label_fash) 

In [ ]:
np.random.seed(1002)
# Parameters
res_param=[]
res_bias=[]
param=[]
bias=[]
mask=[]


res_param.append(np.random.randn(300,784)/100) 


#res_param.append(np.random.randn(300,300)/100) 





res_bias.append(np.random.randn(300)/100) 


param.append(np.random.randn(300,300)/100) 
param.append(np.random.randn(10,300)/100 ) 
bias.append(np.random.randn(300)/100) 
bias.append(np.random.randn(10)/100) 



In [ ]:
param=[]
bias=[]
param.append(np.random.randn(300,300)/100) 
param.append(np.random.randn(10,300)/100 ) 
bias.append(np.random.randn(300)/100) 
bias.append(np.random.randn(10)/100) 


In [ ]:
# it seems like the LR has to be very very small 0.001 tends to have better transfer of learning. 
#Also removed the Relu on the input layer.
ctr=0
start_time = timeit.default_timer()
for epochs in range(1,50): 
  for samples in range(50):  
    dparam,dbias=jit(v_gradient)(train_data[ctr:ctr+1000],train_label[ctr:ctr+1000],mask,res_param,res_bias,param,bias) 
    #prune resevoir 
    _,r1,r2=v_forward(train_data[ctr:ctr+1000],mask,res_param,res_bias,param,bias) 
    param,bias=jit(update_weights)(param,bias,dparam,dbias,lr=0.001) 
    ctr=ctr+1000
    if ctr>40000: 
      ctr=0
    #if epochs<10:
      #mask[0]=mask[0]-0.1
      #mask[1]=mask[1]-0.1
     # print('pruning')
      #mask[0]=(plasticity)(r1,mask[0])
      #print('pruning')
     # 3mask[1]=(plasticity)(r2,mask[1])
    
                       
   
  print(NLL_loss(train_data[0],train_label[0],mask,res_param,res_bias,param,bias)) 
  pred1,_,_=jit(v_forward)(train_data[40000:40100],mask,res_param,res_bias,param,bias)
  pred1=jnp.argmax(pred1,axis=1)

  target1=jnp.argmax(train_label[40000:40100],axis=1)
  print (f'epoch: ', epochs+1)
  print(f'Validation Accuracy: ', len(jnp.where(pred1 == target1)[0])/100*100)
elapsed = timeit.default_timer() - start_time
print (f'Total time: ' ,elapsed)

2.305964
epoch:  2
Validation Accuracy:  45.0
2.7935731
epoch:  3
Validation Accuracy:  45.0
2.4027393
epoch:  4
Validation Accuracy:  55.00000000000001
3.042884
epoch:  5
Validation Accuracy:  56.00000000000001
3.82938
epoch:  6
Validation Accuracy:  54.0
4.075954
epoch:  7
Validation Accuracy:  44.0
3.2687657
epoch:  8
Validation Accuracy:  57.99999999999999
3.9132547
epoch:  9
Validation Accuracy:  59.0
3.820639
epoch:  10
Validation Accuracy:  64.0
3.2748387
epoch:  11
Validation Accuracy:  61.0
3.6225832
epoch:  12
Validation Accuracy:  65.0
2.8224645
epoch:  13
Validation Accuracy:  56.99999999999999
3.7518213
epoch:  14
Validation Accuracy:  61.0
3.6586735
epoch:  15
Validation Accuracy:  63.0
3.4210513
epoch:  16
Validation Accuracy:  65.0
4.282209
epoch:  17
Validation Accuracy:  65.0
3.4008708
epoch:  18
Validation Accuracy:  66.0
3.4223416
epoch:  19
Validation Accuracy:  65.0
4.336059
epoch:  20
Validation Accuracy:  60.0
3.6527317
epoch:  21
Validation Accuracy:  63.0
4.42

In [ ]:
#store params 
param_mnist = param
bias_mnist=bias

In [ ]:
#pop layer.
res_param.append(param[0])
res_bias.append(bias[0])

In [ ]:
#Test: 
pred_1,_,_=v_forward(test_data,mask,res_param,res_bias,param,bias)
pred_1=jnp.argmax(pred_1,axis=1)
target_1=jnp.argmax(test_label,axis=1)
print(f'Test Accuracy:',len(jnp.where(pred_1 == target_1)[0])/10000 *100)

Test Accuracy: 68.04


In [ ]:
#Test: 
pred_1,_,_=v_forward(test_data_fash,mask,res_param[0:3],res_bias[0:3],param_mnist,bias_mnist)
pred_1=jnp.argmax(pred_1,axis=1)
target_1=jnp.argmax(test_label_fash,axis=1)
print(f'Test Accuracy:',len(jnp.where(pred_1 == target_1)[0])/10000 *100)

In [ ]:
len(res_param)

1

In [ ]:
#For fashion MNIST
#30 epochs and 30 epochs %85.32
#30 epochs, push, reinitialize 30 epochs. %85.45
#30 epochs, push, reinitialize 30 epochs, reinitialize, 30 epochs %85.46
#So they both hit aroudn 85% within those epochs range. 


#Times = first 30 --> 12.24 seconds second 30-->12.57.

#Times = first 30 --> 12.01 seconds , second 30-->12.199

#Complexity is O(n) 


In [ ]:
#For CIAFAR-10 
#30 epochs and 30 epochs : 40%
#30 epochs pop 30 epochs : 38.31%
#30 epochs pop 30 eopchs pop 30 epochs : 40.12999%
#no change. 